<a href="https://colab.research.google.com/github/sherif17/Hand_Signature_Detection_Verification/blob/main/Hand_Signature_detection_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Imports**

In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
import os
import csv

from tqdm.auto import tqdm
import shutil as sh
from sklearn import preprocessing, model_selection

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15529, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 15529 (delta 49), reused 118 (delta 43), pack-reused 15393
Receiving objects: 100% (15529/15529), 14.59 MiB | 17.17 MiB/s, done.
Resolving deltas: 100% (10573/10573), done.


In [ ]:
%%capture
!pip install GitPython
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
!gdown https://drive.google.com/u/0/uc?id=1gew1zSfSZKiUKGPGc3bpGXbO03HvE9-_
!gdown --fuzzy https://drive.google.com/file/d/1v79mdeZuSY_P7NfK8oNNM0n7qsRlaEG_/view

## **Reading DataSet**

In [ ]:
# Define the path to the dataset folder
dataset_path = '/content/drive/MyDrive/Colab Notebooks/AI-ITI/Computer_vision/Assignment_Three_Detection/SignatureObjectDetection'

## **Annotating DataSet**

In [ ]:
def get_data_annotate(folder_path, csv_file, img_path):

    # List to store the annotations
    annotations = []
    # Iterate over the text files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            with open(os.path.join(folder_path, file_name), 'r') as f:
                # Read all the lines in the file
                lines = f.readlines()
                # Process each line
                for line in lines:
                    # Extract the bbox coordinates from the line
                    bbox = line.strip().split(',')
                    # Add the annotation to the list
                    xmin, ymin, xmax, ymax = list(map(int,[bbox[0], bbox[1], bbox[2], bbox[3]]))
                    xcenter = (xmin+xmax)/2
                    ycenter = (ymin+ymax)/2
                    w = xmax-xmin
                    h = ymax-ymin

                    xcenter /= 1000
                    ycenter /= 1000
                    w /= 1000
                    h /= 1000
                    annotations.append([file_name.replace('.txt', '.tif'), 0 ,xcenter,ycenter, w, h])

    # Write the annotations to the CSV file
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        #writer.writerow(['filename', 'class_name','x1', 'y1', 'x2', 'y2'])
        writer.writerows(annotations)
        

In [ ]:
# Set the directory path where the text files are located
train_truth_path = os.path.join(dataset_path,'TrainGroundTruth')
train_image_path = os.path.join(dataset_path,'TrainImages')
test_truth_path = os.path.join(dataset_path,'TestGroundTruth')
test_image_path = os.path.join(dataset_path,'TestImages')


get_data_annotate(train_truth_path, "train_data.csv",train_image_path)
get_data_annotate(test_truth_path, "test_data.csv",test_image_path)

In [ ]:
df = pd.read_csv('train_data.csv',names=['filename','label','x_center','y_center','width','height'])
df.head(1)

,filename,label,x_center,y_center,width,height
0,34.tif,0,0.694,0.66,0.438,0.08


In [ ]:
dest_root_path = '/content/dataset'
dest_img_path = os.path.join(dest_root_path,'images')
dest_label_path = os.path.join(dest_root_path,'labels')
dest_train_img_path = os.path.join(dest_root_path,'images/train')
dest_train_label_path = os.path.join(dest_root_path, 'labels/train')

try:
    os.mkdir(dest_root_path)
    os.mkdir(dest_img_path)
    os.mkdir(dest_label_path)
    os.mkdir(dest_train_img_path)
    os.mkdir(dest_train_label_path)
except:
    pass

In [ ]:
for filename in df.filename.unique():
    row = df[df.filename == filename][['label','x_center','y_center','width','height']]
    sh.copy(os.path.join(train_image_path,filename), os.path.join(dest_train_img_path,filename))
    np.savetxt(os.path.join(dest_train_label_path,filename.replace('tif','txt')), np.array(row), fmt=["%d", "%f", "%f", "%f", "%f"])

In [ ]:
df_test = pd.read_csv('test_data.csv',names=['filename','label','x_center','y_center','width','height'])
df_test.head(1)

,filename,label,x_center,y_center,width,height
0,764.tif,0,0.668,0.844,0.324,0.066


In [ ]:
dest_val_img_path = os.path.join(dest_root_path,'images/val')
dest_val_label_path = os.path.join(dest_root_path, 'labels/val')
try:
    os.mkdir(dest_val_img_path)
    os.mkdir(dest_val_label_path)
except:
    pass

In [ ]:
for filename in df_test.filename.unique():
    row = df_test[df_test.filename == filename][['label','x_center','y_center','width','height']]
    sh.copy(os.path.join(test_image_path,filename), os.path.join(dest_val_img_path,filename))
    np.savetxt(os.path.join(dest_val_label_path,filename.replace('tif','txt')), np.array(row), fmt=["%d", "%f", "%f", "%f", "%f"])

In [ ]:
# create dataset yaml
import yaml

# Define the file path
file_path = "dataset/structure.yaml"

# Define the file contents as a dictionary
file_contents = {
    "path": "../dataset",
    "train": "images/train",
    "val": "images/val",
    "names": {
        0: "Signature"
    }
}

# Write the file
with open(file_path, "w") as f:
    yaml.dump(file_contents, f)

In [ ]:
#defult iou = 0.65
! python yolov5/train.py --img 600 --batch 16 --epochs 50 --data dataset/structure.yaml --weights yolov5/yolov5x.pt --hyp yolov5/data/hyps/hyp.scratch-low.yaml

train: weights=yolov5/yolov5x.pt, cfg=, data=dataset/structure.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=600, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-145-g94714fe Python-3.9.16 torch-2.0.0+cu118 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, io

In [ ]:
! python yolov5/val.py --batch 16 --data dataset/structure.yaml --task val --weights yolov5/runs/train/exp/weights/best.pt --name best_val_

val: data=dataset/structure.yaml, weights=['yolov5/runs/train/exp/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=best_val_, exist_ok=False, half=False, dnn=False
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-145-g94714fe Python-3.9.16 torch-2.0.0+cu118 CPU

Traceback (most recent call last):
  File "/content/yolov5/val.py", line 409, in <module>
    main(opt)
  File "/content/yolov5/val.py", line 380, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/val.py", line 143, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/content/yolov5/models/common

In [ ]:
DIR = 'yolov5/runs/val/best_val_2'
for img_name in os.listdir(DIR):
    img = cv2.imread(DIR + '/' + img_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot()
    ax.imshow(img)
    print(img_name)
    plt.show()

FileNotFoundError: ignored